# G2ConfigMgr

The G2ConfigMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database,
and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID
that points to one of those configurations.
That configuration ID will then be the shared global config.

## Prepare environment

In [ ]:
import com.senzing.g2.engine.G2ConfigMgr;
import com.senzing.g2.engine.G2ConfigMgrJNI;
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
RenderJSON(senzingConfigJson);

## G2Config

### Create G2Config instance

Create an instance of `G2Config` for use with managing configuration.

In [ ]:
G2Config g2Config = new G2ConfigJNI();

String moduleName = "ExampleG2Config";
boolean verboseLogging = true;
int returnCode = g2Config.initV2(moduleName, senzingConfigJson, verboseLogging);
System.out.print(returnCode);

## G2ConfigMgr

The G2ConfigMgr API is used to add specific JSON configurations to the database,
so that they may be shared across remote systems.
Such configurations are added to the database, and a configuration ID is created for each of them.
The system may then be configured with a specific configuratin ID that points to one of those configurations.
That configuration ID will then be the shared global config.

### G2ConfigMgr Initialization

To start using G2ConfigMgr, you must first create and initialize an instance of the config manager. This should be done once per process.

Create a new instance of the config manager and assign it to a variable. Then, call the appropriate initialization method (such as initV2) to initialize the config manager.

During the call, the initialization methods accept the following parameters:

- **moduleName:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzingConfigJson:** A JSON document containing system parameters (see the section called "Initialization Parameters")
- **verboseLogging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)
Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
G2ConfigMgr g2ConfigMgr = new G2ConfigMgrJNI();

String moduleName = "ExampleG2ConfigMgr";
boolean verboseLogging = true;
int returnCode = g2ConfigMgr.initV2(moduleName, senzingConfigJson, verboseLogging);
g2ConfigMgr.getLastException();
System.out.print(returnCode);

### Managing configurations

Configuration JSON documents may be added to the datastore, so that they can be shared across remote systems.
The `create()` method makes a new in-memory configuration from the `g2config.json` file.

In [ ]:
long configHandle = g2Config.create();

Save the in-memory configuration to a string buffer.

In [ ]:
StringBuffer configStringBuffer = new StringBuffer();
int returnCode = g2Config.save(configHandle, configStringBuffer);
if (returnCode != 0)
    System.out.print(g2Config.getLastException());
else
    RenderJSON(configStringBuffer);

#### addConfig

Use `addConfig()` to add a configuration JSON document to the data repository.

The `addConfig()` function accepts the following parameters as input:

- **configStr:** The configuration JSON document.
- **configComments:** A free-form string of comments describing the configuration document.
- **configID:** The returned configID for the new config document registered in the data store.
The function returns "0" upon success.

In [ ]:
String configStr = configStringBuffer.toString();
String timeStamp = new java.text.SimpleDateFormat("yyyy.MM.dd.HH.mm.ss").format(new java.util.Date());
String configComments = "senzing-G2ConfigMgr-reference.ipynb added at " + timeStamp;
Result<Long> configID = new Result<Long>();

int returnCode = g2ConfigMgr.addConfig(configStr, configComments, configID);
System.out.print("CONFIG ID: " + configID.getValue());

#### getConfigList

Use `getConfigList()` to retrieve a list of the configuration JSON documents contained in the data repository.
The `getConfigList()` function has no input arguments, and various arguments used to return response documents.

In [ ]:
StringBuffer response = new StringBuffer();

int returnCode = g2ConfigMgr.getConfigList(response);
RenderJSON(response);

#### getConfig
Use `getConfig()` to retrieve a specific configuration JSON document from the data repository.

The `getConfig()` function accepts the following parameters as input:

- **configID:** The configID for the config document that you wish to retrieve.
They also have various arguments used to return response documents.

In [ ]:
StringBuffer response = new StringBuffer();

int returnCode = g2ConfigMgr.getConfig(configID.getValue(), response);
RenderJSON(response);

### Default configurations

Multiple configuration JSON documents may be added to the datastore. Each of them may be referenced to start the engine and other API's with different configurations.

In order to specify that one of the configuration documents should be the global shared config, the config ID for that configuration document must be set as the default config ID.

#### setDefaultConfigID

Use `setDefaultConfigID()` to set the default configuration JSON document in the data repository.

The `setDefaultConfigID()` function accepts the following parameters as input:

- **configID:** The configuration ID for a configuration JSON document previously added to the database.

The function returns "0" upon success.

In [ ]:
int returnCode = g2ConfigMgr.setDefaultConfigID(configID.getValue());
System.out.print(returnCode);

#### getDefaultConfigID

Use `getDefaultConfigID()` to retrieve a specific configuration JSON document from the data repository.
The `getDefaultConfigID()` function accepts the following parameters as input:

- **configID:** Returns the configID for the current default configuration, or 0 if none is set.

In [ ]:
Result<Long> configID = new Result<Long>();
int returnCode = g2ConfigMgr.getDefaultConfigID(configID);
if(returnCode!=0)
    System.out.print(g2Config.getLastException());
else
    System.out.print(configID.getValue());